In [80]:
import requests
import re
import urllib.request
import pandas as pd
import sqlite3
import json

import csv
from bs4 import BeautifulSoup

## get a list of PA state park/forest from wiki
121 state parks, and 20 state forests

In [3]:
# get the url
url_park = 'https://en.wikipedia.org/wiki/List_of_Pennsylvania_state_parks'
response_park = requests.get(url_park)
print(response.status_code) 

soup = BeautifulSoup(response_park.text,"html.parser")

200


In [4]:
# find the table in need: first sortable table 
table = soup.findAll('table',{"class":"wikitable sortable"})[0]
# get table rows
tr = table.findAll(['tr']) # a total of 121 parks [& header]


# write info from the table to a csv file
csvFile = open("state_park_PA.csv",'wt',newline='', encoding='utf-8')
writer = csv.writer(csvFile)  
try:   
    for cell in tr:
        th = cell.find_all('th')
        th_data = [col.text.strip('\n') for col in th]
        td = cell.find_all('td')
        row = [i.text.replace('\n','') for i in td]
        writer.writerow(th_data+row)      
        
finally:   
    csvFile.close()

### repeat for state forest

In [5]:
url_forest = 'https://en.wikipedia.org/wiki/List_of_Pennsylvania_state_forests'
response_forest = requests.get(url_forest)
print(response_forest.status_code)

soup = BeautifulSoup(response_forest.text,"html.parser")

table = soup.findAll('table',{"class":"wikitable sortable"})[0]
tr = table.findAll(['tr']) # a total of 20 forests [& header]
csvFile = open("state_forest_PA.csv",'wt',newline='', encoding='utf-8')
writer = csv.writer(csvFile)  
try:   
    for cell in tr:
        th = cell.find_all('th')
        th_data = [col.text.strip('\n') for col in th]
        td = cell.find_all('td')
        row = [i.text.replace('\n','') for i in td]
        writer.writerow(th_data+row)      
        
finally:   
    csvFile.close()

200


In [17]:
df_forest = pd.read_csv('state_forest_PA.csv')
df_forest

,State Forest Name,County,Areaacres (ha),Founded,Remarks
0,Bald Eagle,"Centre, Clinton, Mifflin, Union, & Snyder","193,424 acres (78,280 ha)",NaN,NaN
1,Buchanan,"Bedford, Franklin, & Fulton","75,000 acres (30,350 ha)",NaN,NaN
2,Clear Creek,"Forest, Jefferson, & Venango","13,266 acres (5,369 ha)","1919, as Kittanning State Forest",Name changed 2007
3,Cornplanter,"Crawford, Forest, & Warren","1,256 acres (508 ha)",NaN,NaN
4,Delaware,Pike & Monroe,"83,519 acres (33,799 ha)",NaN,NaN
5,Elk,"Cameron, Clinton, Elk, McKean, & Potter","200,000 acres (80,940 ha)",1900,NaN
6,Forbes,"Fayette, Somerset, & Westmoreland","over 50,000 acres (20,230 ha)",NaN,NaN
7,Gallitzin,"Bedford, Cambria, Indiana, & Somerset","15,336 acres (6,206 ha)",NaN,NaN
8,Loyalsock,"Bradford, Lycoming, & Sullivan","114,552 acres (46,360 ha)","July 1, 2005",Replaced Wyoming State Forest
9,Michaux,"Adams, Cumberland, & Franklin","over 85,000 acres (34,400 ha)",NaN,NaN


## Search for these places with google Place API
bash fetch_state_park.sh 

In [23]:
con = sqlite3.connect("state_forest_PA_google_place_id.db")
df = pd.read_sql_query("SELECT id, name, address FROM state_forest", con)
con.close()
df

,id,name,address
0,ChIJVdp4ZCi8zokR-BN2PkJVZAQ,Bald Eagle State Forest,"Bald Eagle State Forest, Walker Township, PA 1..."
1,ChIJp6qqqvpbyokRlThh6d1zxHo,Buchanan State Forest,"Clearville, PA 15535, United States"
2,ChIJG9pKq3nezIkRf8e_mHrUZ0s,Clear Creek State Forest,"Sigel, PA 15860, United States"
3,ChIJvyueE8pEzYkRoVBwUV2-kuw,Cornplanter State Forest,"North Warren, PA 16365, United States"
4,ChIJU1VVVZWfxIkRhUhK_7Y6xJo,Delaware State Forest,"Dingmans Ferry, PA 18328, United States"
5,ChIJcRuHo6fKzYkRj01hdSwOfu4,Elk State Forest,"Elk State Forest, Benezette Township, PA 15868..."
6,ChIJM4TClPcny4kRhVB49yVSEI0,Forbes State Forest,"Forbes State Forest, Cook Township, PA 15677, USA"
7,ChIJa4nSqlYGy4kREMPuwlu9T78,Gallitzin State Forest,"Gallitzin State Forest, Ogle Township, PA 1596..."
8,ChIJOxLy42Tsz4kRxcGLnp5ff0w,Loyalsock State Forest,"Loyalsock State Forest, Fox Township, PA 18616..."
9,ChIJk472rC-hyYkRUHOh-AkNBvQ,Michaux State Forest,"10099 Lincoln Way E, Fayetteville, PA 17222, U..."


In [26]:
con = sqlite3.connect("state_park_PA_google_place_id.db")
df_park = pd.read_sql_query("SELECT id, name, address FROM state_park", con)
con.close()
df_park

,id,name,address
0,ChIJFbEYjJqUNIgRPLxC2ZWBxe4,Allegheny Islands State Park,"Cheswick, PA 15024, United States"
1,ChIJqV69o1LQxIkRcH7deU6CJFg,Archbald Pothole State Park,"960 Scranton Carbondale Hwy, Archbald, PA 1840..."
2,ChIJPUAh3FWRzokRTusj8NUp8dk,Bald Eagle State Park,"149 Main Park Rd, Howard, PA 16841, United States"
3,ChIJF9CkZ8FRxIkRH3KWMSBmvBY,Beltzville State Park,"2950 Pohopoco Dr, Lehighton, PA 18235, United ..."
4,ChIJ0Xs1BMSnzYkR6GLE86aPuT8,Bendigo State Park,"533 Johnsonburg Rd, St Marys, PA 15857, United..."
...,...,...,...
116,ChIJdfywo6xXwYkRuS111bIPKqU,Washington Crossing Historic Park,"1112 River Rd, Washington Crossing, PA 18977, ..."
117,ChIJk00-rKOrzokRS_4BHwJvaUM,Whipple Dam State Park,"3746 Laurel Run Rd, Petersburg, PA 16669, Unit..."
118,ChIJod2hF0Srx4kRNsFIgSQ44JQ,White Clay Creek Preserve,"404 Sharpless Rd, Landenberg, PA 19350, United..."
119,ChIJaeKPQD_zz4kRy4c8TvnwGIg,Worlds End State Park,"82 Cabin Bridge Rd, Forksville, PA 18616, Unit..."


## Combine the three database preparing for scraping

In [27]:
all_parks_con = sqlite3.connect('data/selected_pine_grove_cape_may_r2k_places_20210316_144723.db')
all_parks = pd.read_sql_query("SELECT id, name, address FROM task_list", all_parks_con)
all_parks_con.close()
all_parks

,id,name,address
0,ChIJAWkAqNL1t4kRlm4slspOSXo,Quiet Waters Park,"600 Quiet Waters Park Rd, Annapolis, MD 21403,..."
1,ChIJzcow6Xb1t4kRQVE7s1AWWr8,Hillsmere Shores Community Beach,"101 W Bay View Dr, Annapolis, MD 21403, USA"
2,ChIJ_bk5K1z1t4kRQNverIUOVko,Quiet Waters Dog Beach,"1701-1799 Quiet Waters Park Rd, Annapolis, MD ..."
3,ChIJ_-2cQEX1t4kRoiCrmNMbrQ8,Quiet Waters Dog Park,"600 Quiet Waters Park Rd, Annapolis, MD 21403,..."
4,ChIJQWcsgIn2t4kRCL0Ub363FyI,Pip Moyer Recreation Center (Annapolis Recreat...,"273 Hilltop Ln, Annapolis, MD 21403, USA"
...,...,...,...
1651,ChIJfVzC4e1EwYkR7CX7hHL8w-I,Township Park,"Columbus, NJ 08022, USA"
1652,ChIJN_dm4uVbwYkRmsPTaQhpCsI,Abbott Marshlands,"Bordentown, NJ 08505, USA"
1653,ChIJd9ol6rtbwYkR6v7KxxO7Z6k,Scenic Overlook,"Hamilton Township, NJ 08611, USA"
1654,ChIJ8VFM3PLjw4kRWHo927zq9ZM,Carson Road Woods,"Carson Rd, Princeton, NJ 08540, USA"


In [28]:
state_forest_con = sqlite3.connect('state_forest_PA_google_place_id.db')
state_forest = pd.read_sql_query("SELECT id, name, address FROM state_forest", state_forest_con)
state_forest_con.close()
state_forest

,id,name,address
0,ChIJVdp4ZCi8zokR-BN2PkJVZAQ,Bald Eagle State Forest,"Bald Eagle State Forest, Walker Township, PA 1..."
1,ChIJp6qqqvpbyokRlThh6d1zxHo,Buchanan State Forest,"Clearville, PA 15535, United States"
2,ChIJG9pKq3nezIkRf8e_mHrUZ0s,Clear Creek State Forest,"Sigel, PA 15860, United States"
3,ChIJvyueE8pEzYkRoVBwUV2-kuw,Cornplanter State Forest,"North Warren, PA 16365, United States"
4,ChIJU1VVVZWfxIkRhUhK_7Y6xJo,Delaware State Forest,"Dingmans Ferry, PA 18328, United States"
5,ChIJcRuHo6fKzYkRj01hdSwOfu4,Elk State Forest,"Elk State Forest, Benezette Township, PA 15868..."
6,ChIJM4TClPcny4kRhVB49yVSEI0,Forbes State Forest,"Forbes State Forest, Cook Township, PA 15677, USA"
7,ChIJa4nSqlYGy4kREMPuwlu9T78,Gallitzin State Forest,"Gallitzin State Forest, Ogle Township, PA 1596..."
8,ChIJOxLy42Tsz4kRxcGLnp5ff0w,Loyalsock State Forest,"Loyalsock State Forest, Fox Township, PA 18616..."
9,ChIJk472rC-hyYkRUHOh-AkNBvQ,Michaux State Forest,"10099 Lincoln Way E, Fayetteville, PA 17222, U..."


In [29]:
state_park_con = sqlite3.connect('state_park_PA_google_place_id.db')
state_park = pd.read_sql_query("SELECT id, name, address FROM state_park", state_park_con)
state_park_con.close()
state_park

,id,name,address
0,ChIJFbEYjJqUNIgRPLxC2ZWBxe4,Allegheny Islands State Park,"Cheswick, PA 15024, United States"
1,ChIJqV69o1LQxIkRcH7deU6CJFg,Archbald Pothole State Park,"960 Scranton Carbondale Hwy, Archbald, PA 1840..."
2,ChIJPUAh3FWRzokRTusj8NUp8dk,Bald Eagle State Park,"149 Main Park Rd, Howard, PA 16841, United States"
3,ChIJF9CkZ8FRxIkRH3KWMSBmvBY,Beltzville State Park,"2950 Pohopoco Dr, Lehighton, PA 18235, United ..."
4,ChIJ0Xs1BMSnzYkR6GLE86aPuT8,Bendigo State Park,"533 Johnsonburg Rd, St Marys, PA 15857, United..."
...,...,...,...
116,ChIJdfywo6xXwYkRuS111bIPKqU,Washington Crossing Historic Park,"1112 River Rd, Washington Crossing, PA 18977, ..."
117,ChIJk00-rKOrzokRS_4BHwJvaUM,Whipple Dam State Park,"3746 Laurel Run Rd, Petersburg, PA 16669, Unit..."
118,ChIJod2hF0Srx4kRNsFIgSQ44JQ,White Clay Creek Preserve,"404 Sharpless Rd, Landenberg, PA 19350, United..."
119,ChIJaeKPQD_zz4kRy4c8TvnwGIg,Worlds End State Park,"82 Cabin Bridge Rd, Forksville, PA 18616, Unit..."


### dataframe merge

In [42]:
merged_parks = pd.concat((all_parks, state_forest, state_park), axis=0)

In [43]:
task_list = merged_parks.drop_duplicates(subset=['id'])
task_list.shape

(1784, 3)

In [45]:
# task_conn = sqlite3.connect("selected_philly_outdoor_places.db")
# task_list.to_sql('task_list', task_conn)
# task_conn.close()

In [44]:
task_list

,id,name,address
0,ChIJAWkAqNL1t4kRlm4slspOSXo,Quiet Waters Park,"600 Quiet Waters Park Rd, Annapolis, MD 21403,..."
1,ChIJzcow6Xb1t4kRQVE7s1AWWr8,Hillsmere Shores Community Beach,"101 W Bay View Dr, Annapolis, MD 21403, USA"
2,ChIJ_bk5K1z1t4kRQNverIUOVko,Quiet Waters Dog Beach,"1701-1799 Quiet Waters Park Rd, Annapolis, MD ..."
3,ChIJ_-2cQEX1t4kRoiCrmNMbrQ8,Quiet Waters Dog Park,"600 Quiet Waters Park Rd, Annapolis, MD 21403,..."
4,ChIJQWcsgIn2t4kRCL0Ub363FyI,Pip Moyer Recreation Center (Annapolis Recreat...,"273 Hilltop Ln, Annapolis, MD 21403, USA"
...,...,...,...
114,ChIJmxXHK1TIxIkR6GSCKXxJiJw,Varden Conservation Area,"225 Tannery Rd, Lake Ariel, PA 18436, United S..."
115,ChIJVVVVVWXCy4kRrROvaVhWl_U,Warriors Path State Park,"1152 Warriors Path Park Rd, Saxton, PA 16678, ..."
117,ChIJk00-rKOrzokRS_4BHwJvaUM,Whipple Dam State Park,"3746 Laurel Run Rd, Petersburg, PA 16669, Unit..."
119,ChIJaeKPQD_zz4kRy4c8TvnwGIg,Worlds End State Park,"82 Cabin Bridge Rd, Forksville, PA 18616, Unit..."


In [52]:
task_list[task_list.id == 'ChIJbZpQs8qjxokRaNkG7x9tGas']

,id,name,address
912,ChIJbZpQs8qjxokRaNkG7x9tGas,Whites Road Park,"400 Whites Rd, Lansdale, PA 19446, USA"


## Get some features for state system parks/forests
For example: [rating 	user_ratings_total 	geometry 	lat 	lng]

In [104]:
state_park_con = sqlite3.connect('state_park_PA_google_place_id.db')
state_park = pd.read_sql_query("SELECT id, name, address FROM state_park", state_park_con)

raw_content = pd.read_sql_query("SELECT data FROM state_park", state_park_con)
state_park_con.close()

In [66]:
def extract_feature(value, feature, default=None):
    content = json.loads(value)['candidates'][0]
    return content.get(feature, default)

In [71]:
def extract_location(value, default=None):
    content = json.loads(value)['candidates'][0]
    geo = content.get('geometry', default)
    if geo:
        location = geo.get('location', default)
        if location:
            coords = (location['lat'], location['lng'])
    else:
        coords = default
    return coords

In [105]:
state_park['rating'] = raw_content['data'].apply(extract_feature, args=('rating', ))
state_park['user_ratings_total'] = raw_content['data'].apply(extract_feature, args=('user_ratings_total', ))

In [69]:
json.loads(raw_content['data'].iloc[0])['candidates'][0].keys()

dict_keys(['business_status', 'formatted_address', 'geometry', 'icon', 'name', 'photos', 'place_id', 'plus_code', 'rating', 'types', 'user_ratings_total'])

In [106]:
state_park['geometry'] = raw_content['data'].apply(extract_location)
state_park['lat'] = state_park['geometry'].apply(lambda x: x[0])
state_park['lng'] = state_park['geometry'].apply(lambda x: x[1])
state_park

,id,name,address,rating,user_ratings_total,geometry,lat,lng
0,ChIJFbEYjJqUNIgRPLxC2ZWBxe4,Allegheny Islands State Park,"Cheswick, PA 15024, United States",4.4,59.0,"(40.5372947, -79.8216202)",40.537295,-79.821620
1,ChIJqV69o1LQxIkRcH7deU6CJFg,Archbald Pothole State Park,"960 Scranton Carbondale Hwy, Archbald, PA 1840...",3.9,178.0,"(41.5130357, -75.57574509999999)",41.513036,-75.575745
2,ChIJPUAh3FWRzokRTusj8NUp8dk,Bald Eagle State Park,"149 Main Park Rd, Howard, PA 16841, United States",4.6,1607.0,"(41.0226186, -77.645021)",41.022619,-77.645021
3,ChIJF9CkZ8FRxIkRH3KWMSBmvBY,Beltzville State Park,"2950 Pohopoco Dr, Lehighton, PA 18235, United ...",4.6,2649.0,"(40.8650993, -75.627268)",40.865099,-75.627268
4,ChIJ0Xs1BMSnzYkR6GLE86aPuT8,Bendigo State Park,"533 Johnsonburg Rd, St Marys, PA 15857, United...",4.5,112.0,"(41.52818, -78.628941)",41.528180,-78.628941
...,...,...,...,...,...,...,...,...
116,ChIJdfywo6xXwYkRuS111bIPKqU,Washington Crossing Historic Park,"1112 River Rd, Washington Crossing, PA 18977, ...",4.7,1530.0,"(40.2949314, -74.87211309999999)",40.294931,-74.872113
117,ChIJk00-rKOrzokRS_4BHwJvaUM,Whipple Dam State Park,"3746 Laurel Run Rd, Petersburg, PA 16669, Unit...",4.5,283.0,"(40.6829133, -77.8657022)",40.682913,-77.865702
118,ChIJod2hF0Srx4kRNsFIgSQ44JQ,White Clay Creek Preserve,"404 Sharpless Rd, Landenberg, PA 19350, United...",4.8,105.0,"(39.752061, -75.7753259)",39.752061,-75.775326
119,ChIJaeKPQD_zz4kRy4c8TvnwGIg,Worlds End State Park,"82 Cabin Bridge Rd, Forksville, PA 18616, Unit...",4.8,888.0,"(41.4666991, -76.5753685)",41.466699,-76.575368


In [102]:
# extract postal code from formatted_address
def extract_postalcode(value, default=None):
    postal_code = default
    add_str = json.loads(value)['candidates'][0]['formatted_address']
    postalcode_match = r'.+(PA|Pennsylvania).(\d+),'
    match_str = re.search(postalcode_match, add_str)
    if match_str:
        postal_code = match_str.group(2)
    return postal_code
    

In [107]:
state_park['postal_code'] = raw_content['data'].apply(extract_postalcode)

In [108]:
state_park

,id,name,address,rating,user_ratings_total,geometry,lat,lng,postal_code
0,ChIJFbEYjJqUNIgRPLxC2ZWBxe4,Allegheny Islands State Park,"Cheswick, PA 15024, United States",4.4,59.0,"(40.5372947, -79.8216202)",40.537295,-79.821620,15024
1,ChIJqV69o1LQxIkRcH7deU6CJFg,Archbald Pothole State Park,"960 Scranton Carbondale Hwy, Archbald, PA 1840...",3.9,178.0,"(41.5130357, -75.57574509999999)",41.513036,-75.575745,18403
2,ChIJPUAh3FWRzokRTusj8NUp8dk,Bald Eagle State Park,"149 Main Park Rd, Howard, PA 16841, United States",4.6,1607.0,"(41.0226186, -77.645021)",41.022619,-77.645021,16841
3,ChIJF9CkZ8FRxIkRH3KWMSBmvBY,Beltzville State Park,"2950 Pohopoco Dr, Lehighton, PA 18235, United ...",4.6,2649.0,"(40.8650993, -75.627268)",40.865099,-75.627268,18235
4,ChIJ0Xs1BMSnzYkR6GLE86aPuT8,Bendigo State Park,"533 Johnsonburg Rd, St Marys, PA 15857, United...",4.5,112.0,"(41.52818, -78.628941)",41.528180,-78.628941,15857
...,...,...,...,...,...,...,...,...,...
116,ChIJdfywo6xXwYkRuS111bIPKqU,Washington Crossing Historic Park,"1112 River Rd, Washington Crossing, PA 18977, ...",4.7,1530.0,"(40.2949314, -74.87211309999999)",40.294931,-74.872113,18977
117,ChIJk00-rKOrzokRS_4BHwJvaUM,Whipple Dam State Park,"3746 Laurel Run Rd, Petersburg, PA 16669, Unit...",4.5,283.0,"(40.6829133, -77.8657022)",40.682913,-77.865702,16669
118,ChIJod2hF0Srx4kRNsFIgSQ44JQ,White Clay Creek Preserve,"404 Sharpless Rd, Landenberg, PA 19350, United...",4.8,105.0,"(39.752061, -75.7753259)",39.752061,-75.775326,19350
119,ChIJaeKPQD_zz4kRy4c8TvnwGIg,Worlds End State Park,"82 Cabin Bridge Rd, Forksville, PA 18616, Unit...",4.8,888.0,"(41.4666991, -76.5753685)",41.466699,-76.575368,18616


In [103]:
# repeat for state forest
state_forest_con = sqlite3.connect('state_forest_PA_google_place_id.db')
state_forest = pd.read_sql_query("SELECT id, name, address FROM state_forest", state_forest_con)

raw_content = pd.read_sql_query("SELECT data FROM state_forest", state_forest_con)
state_forest_con.close()

state_forest['rating'] = raw_content['data'].apply(extract_feature, args=('rating', ))
state_forest['user_ratings_total'] = raw_content['data'].apply(extract_feature, args=('user_ratings_total', ))
state_forest['geometry'] = raw_content['data'].apply(extract_location)
state_forest['lat'] = state_forest['geometry'].apply(lambda x: x[0])
state_forest['lng'] = state_forest['geometry'].apply(lambda x: x[1])
state_forest['postal_code'] = raw_content['data'].apply(extract_postalcode)

state_forest

,id,name,address,rating,user_ratings_total,geometry,lat,lng,postal_code
0,ChIJVdp4ZCi8zokR-BN2PkJVZAQ,Bald Eagle State Forest,"Bald Eagle State Forest, Walker Township, PA 1...",4.6,11,"(40.8878414, -77.65610889999999)",40.887841,-77.656109,16823
1,ChIJp6qqqvpbyokRlThh6d1zxHo,Buchanan State Forest,"Clearville, PA 15535, United States",4.6,101,"(39.7866444, -78.5512595)",39.786644,-78.551260,15535
2,ChIJG9pKq3nezIkRf8e_mHrUZ0s,Clear Creek State Forest,"Sigel, PA 15860, United States",3.5,2,"(41.320606, -79.0557567)",41.320606,-79.055757,15860
3,ChIJvyueE8pEzYkRoVBwUV2-kuw,Cornplanter State Forest,"North Warren, PA 16365, United States",0.0,0,"(41.8742303, -79.13325669999999)",41.874230,-79.133257,16365
4,ChIJU1VVVZWfxIkRhUhK_7Y6xJo,Delaware State Forest,"Dingmans Ferry, PA 18328, United States",4.5,4,"(41.2286853, -74.9910112)",41.228685,-74.991011,18328
5,ChIJcRuHo6fKzYkRj01hdSwOfu4,Elk State Forest,"Elk State Forest, Benezette Township, PA 15868...",4.8,11,"(41.3333946, -78.2914009)",41.333395,-78.291401,15868
6,ChIJM4TClPcny4kRhVB49yVSEI0,Forbes State Forest,"Forbes State Forest, Cook Township, PA 15677, USA",4.3,6,"(40.1295188, -79.18753079999999)",40.129519,-79.187531,15677
7,ChIJa4nSqlYGy4kREMPuwlu9T78,Gallitzin State Forest,"Gallitzin State Forest, Ogle Township, PA 1596...",4.3,3,"(40.2084107, -78.70807479999999)",40.208411,-78.708075,15963
8,ChIJOxLy42Tsz4kRxcGLnp5ff0w,Loyalsock State Forest,"Loyalsock State Forest, Fox Township, PA 18616...",4.8,9,"(41.5109092, -76.72023229999999)",41.510909,-76.720232,18616
9,ChIJk472rC-hyYkRUHOh-AkNBvQ,Michaux State Forest,"10099 Lincoln Way E, Fayetteville, PA 17222, U...",4.9,120,"(39.9046105, -77.4703395)",39.904610,-77.470339,17222


In [110]:
# stack state_park and state_forest ==> state_sys_park
state_sys_park = pd.concat((state_park, state_forest), axis=0)
state_sys_park

,id,name,address,rating,user_ratings_total,geometry,lat,lng,postal_code
0,ChIJFbEYjJqUNIgRPLxC2ZWBxe4,Allegheny Islands State Park,"Cheswick, PA 15024, United States",4.4,59.0,"(40.5372947, -79.8216202)",40.537295,-79.821620,15024
1,ChIJqV69o1LQxIkRcH7deU6CJFg,Archbald Pothole State Park,"960 Scranton Carbondale Hwy, Archbald, PA 1840...",3.9,178.0,"(41.5130357, -75.57574509999999)",41.513036,-75.575745,18403
2,ChIJPUAh3FWRzokRTusj8NUp8dk,Bald Eagle State Park,"149 Main Park Rd, Howard, PA 16841, United States",4.6,1607.0,"(41.0226186, -77.645021)",41.022619,-77.645021,16841
3,ChIJF9CkZ8FRxIkRH3KWMSBmvBY,Beltzville State Park,"2950 Pohopoco Dr, Lehighton, PA 18235, United ...",4.6,2649.0,"(40.8650993, -75.627268)",40.865099,-75.627268,18235
4,ChIJ0Xs1BMSnzYkR6GLE86aPuT8,Bendigo State Park,"533 Johnsonburg Rd, St Marys, PA 15857, United...",4.5,112.0,"(41.52818, -78.628941)",41.528180,-78.628941,15857
...,...,...,...,...,...,...,...,...,...
15,ChIJF2xyqr4Hz4kR60A4AhiwE70,Tiadaghton State Forest,"Tiadaghton State Forest, Washington Township, ...",5.0,5.0,"(41.1611901, -77.0391343)",41.161190,-77.039134,17752
16,ChIJC3wc8swjzokRGv8rNWf02-I,Tioga State Forest,"Tioga State Forest, Gaines Township, PA 16901,...",4.9,7.0,"(41.7770152, -77.5138711)",41.777015,-77.513871,16901
17,ChIJhY25_FM_yYkR66j8jaaaJxQ,Tuscarora State Forest,"Tuscarora State Forest, Jackson Township, PA 1...",4.9,9.0,"(40.3611926, -77.5555447)",40.361193,-77.555545,17006
18,ChIJ5WjwhyL9xYkRkriPfC9HGko,Weiser State Forest,"Appalachian Trail, Pine Grove, PA 17963, Unite...",4.5,20.0,"(40.512814, -76.3440049)",40.512814,-76.344005,17963


#### save a table with these basic information about state system parks

In [114]:
# info_conn = sqlite3.connect("philly_state_parks_info.db")
# state_sys_park.drop(columns=['geometry']).to_sql('info', info_conn)
# info_conn.close()